# Digital inverter with OpenLane

```
Copyright 2022 Google LLC.
SPDX-License-Identifier: Apache-2.0
```

Run a simple digital inverter design thru the [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/) GDS to RTL flow targeting the [open source SKY130 PDK](https://github.com/google/skywater-pdk/).

In [4]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''

import os
import pathlib

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version}
!python -m pip install gdstk gdstk
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}


Empty environment created at prefix: /content/conda-env

Pinned packages:
  - python 3.7*


Transaction

  Prefix: /content/conda-env

  Updating specs:

   - openlane=*
   - open_pdks.sky130a=*


  Package                                                Version  Build                 Channel         Size
──────────────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────────────────────────────

  + open_pdks.sky130a                         1.0.460_0_gfdb1863  20231104_052339       litex-hub     Cached
  + _libgcc_mutex                                            0.1  main                  main          Cached
  + libstdcxx-ng                                          11.2.0  h1234567_1            main          Cached
  + ld_impl_linux-64                                        2.38  h1181459_1            main          Cached
  + ca-certificates     

## Write verilog

In [9]:
%%writefile sqdet.v


module sqdet (
    input wire din,
    input wire clk,
    input wire rst,
    output reg dout
);

reg [3:0] ps;
parameter R = 4'b0000, A0 = 4'b0001, A1 = 4'b0010, A2 = 4'b0011,
          A3 = 4'b0100, A4 = 4'b0101, A5 = 4'b0110, A6 = 4'b0111, A7 = 4'b1000;

always @(posedge clk or negedge rst) begin
    if (!rst) begin
        ps <= R;
        dout = 0;
    end else begin
        case (ps)
            R: begin
                dout = 0;
                if (din == 0) ps = A1;
                else ps = A0;
            end
            A0: begin
                dout = 0;
                if (din == 0) ps = A4;
                else ps = A0;
            end
            A1: begin
                dout = 0;
                if (din == 0) ps = A1;
                else ps = A2;
            end
            A2: begin
                dout = 0;
                if (din == 0) ps = A3;
                else ps = A0;
            end
            A3: begin
                dout = 0;
                if (din == 0) ps = A6;
                else ps = A2;
            end
            A4: begin
                dout = 0;
                if (din == 0) ps = A5;
                else ps = A2;
            end
            A5: begin
                dout = 0;
                if (din == 0) ps = A1;
                else ps = A7;
            end
            A6: begin
                dout = 1;
                if (din == 0) ps = A1;
                else ps = A7;
            end
            A7: begin
                dout = 1;
                if (din == 0) ps = A3;
                else ps = A0;
            end
            default: begin
                dout = 0;

            end
        endcase
    end
end

endmodule


Overwriting sqdet.v


## Write configuration

[Documentation](https://openlane.readthedocs.io/en/latest/reference/configuration.html)

In [5]:
%%writefile config.json
{
    "DESIGN_NAME": "sqdet",
    "VERILOG_FILES": "dir::sqdet.v",
    "RUN_CTS": 0,
    "CLOCK_PORT": "1",
    "PL_RANDOM_GLB_PLACEMENT": true,
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 34.5 57.12",
    "PL_TARGET_DENSITY": 0.75,
    "FP_PDN_AUTO_ADJUST": false,
    "FP_PDN_VPITCH": 25,
    "FP_PDN_HPITCH": 25,
    "FP_PDN_VOFFSET": 5,
    "FP_PDN_HOFFSET": 5,
    "GRT_REPAIR_ANTENNAS": 1,
    "RUN_HEURISTIC_DIODE_INSERTION": 0
}

Writing config.json


## Run OpenLane Flow

[OpenLane](https://openlane.readthedocs.io/en/latest/) is an automated [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) to [GDSII](https://en.wikipedia.org/wiki/GDSII) flow based on several components including [OpenROAD](https://theopenroadproject.org/), [Yosys](https://yosyshq.net/yosys/), [Magic](http://www.opencircuitdesign.com/magic/), [Netgen](http://opencircuitdesign.com/netgen/) and custom methodology scripts for design exploration and optimization targeting [open source PDKs](https://github.com/google/open-source-pdks).

![img](https://openlane.readthedocs.io/en/latest/_images/flow_v1.png)

In [ ]:
!pip install libparse

In [10]:
%env PDK=sky130A
!flow.tcl -design .

env: PDK=sky130A
OpenLane 2023.11.03_0_gf4f8dad8-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[INFO]: Using configuration in 'config.json'...
[INFO]: PDK Root: /content/conda-env/share/pdk
[INFO]: Process Design Kit: sky130A
[INFO]: Standard Cell Library: sky130_fd_sc_hd
[INFO]: Optimization Standard Cell Library: sky130_fd_sc_hd
[INFO]: Run Directory: /content/runs/RUN_2024.01.28_06.30.24
[INFO]: Saving runtime environment...
[INFO]: Preparing LEF files for the nom corner...
[INFO]: Preparing LEF files for the min corner...
[INFO]: Preparing LEF files for the max corner...
[WARNING]: PNR_SDC_FILE is not set. It is recommended to write a custom SDC file for the design. Defaulting to BASE_SDC_FILE
[WARNING]: SIGNOFF_SDC_FILE is not set. It is recommended to write a custom SDC file for the design. Defaulting to BASE_SDC_FILE
[INFO]: Running linter (Verilator) (log: 

## Display layout

In [6]:
import pathlib
import gdstk
import IPython.display

gdss = sorted(pathlib.Path('runs').glob('*/results/final/gds/*.gds'))
library = gdstk.read_gds(gdss[-1])
top_cells = library.top_level()
top_cells[0].write_svg('sqdet.svg')
IPython.display.SVG('sqdet.svg')

IndexError: list index out of range

In [ ]:
# Print your STA report here

## Metrics

[Documentation](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html)


In [ ]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(reports[-1])
df.transpose()

,0
design,/content
design_name,inverter
config,RUN_2024.01.20_04.22.01
flow_status,flow completed
total_runtime,0h0m58s0ms
routed_runtime,0h0m33s0ms
(Cell/mm^2)/Core_Util,3044.696139
DIEAREA_mm^2,0.001971
CellPer_mm^2,1522.34807
OpenDP_Util,-1
